In [ ]:
import sys
sys.path.append("../../")

import django
import pylab as pl

from django.db import connections
from firecares.firestation.models import FireDepartment, FireStation
from firecares.utils import dictfetchall
from pandas import DataFrame

%matplotlib inline

django.setup()

print "Number of unmatched fire stations:", FireStation.objects.filter(department__isnull=True).count()

cursor = connections['default'].cursor()

#cursor.execute("enable extension fuzzystrmatch")

queryUnmatchedNearbyStations = """
select a.name as firestation_name, d.geom as firestation_location
from firestation_usgsstructuredata a
inner join firestation_firestation b
  on a.id=b.usgsstructuredata_ptr_id
join firecares_core_address d 
  on b.station_address_id=d.id
where b.department_id is null and d.geom is not null and 
ST_DISTANCE(ST_TRANSFORM(d.geom,900913),ST_TRANSFORM(ST_GEOMFROMTEXT(%s,900913),900913)) * 0.000621371 <= 100 and
levenshtein(a.name,%s) <= 50;
"""
#
queryDepartmentsWithGeo = """
select c.name as department_name, c.id as dep_id, e.city as headquarters_city, 
ST_ASTEXT(ST_TRANSFORM(e.geom,900913)) as headquarters_location
from firestation_firedepartment c 
inner join firecares_core_address e 
  on c.headquarters_address_id=e.id
where e.geom is not null; 
"""


cursor.execute(queryDepartmentsWithGeo)
fireDepartments = dictfetchall(cursor)
df = DataFrame(fireDepartments)
print df.size

# distribution of levenshtein distances
#pl.xlabel("Levenshtein Distance")
#pl.ylabel("Count of records")
totalMatchedStations = 0;
for fireDepartment in fireDepartments:
    print fireDepartment['department_name']
    params = [fireDepartment['headquarters_location'],fireDepartment['department_name'],fireDepartment['headquarters_city']]
    reults = cursor.execute(queryUnmatchedNearbyStations,params)
    nearbyStations = dictfetchall(cursor)
    df = DataFrame(nearbyStations)
    print "Matched Stations:", df.size
    totalMatchedStations += df.size

print "Total Matched Stations:", totalMatchedStations
#for index, row in df.iterrows():
 #   print row['firestation_name'] , row['firestation_address']





Number of unmatched fire stations: 50743
108656
Oklahoma City Fire Department
Matched Stations: 8
McGregor Volunteer Fire Department
Matched Stations: 4
Oak Grove Fire District
Matched Stations: 20
Grantfork Volunteer Fire Department
Matched Stations: 6
Greater Peoria Airport Fire Department
Matched Stations: 40
Ada Fire Department
Matched Stations: 18
Hamel Community Fire Protection District
Matched Stations: 2
Thelma Volunteer Fire Department
Matched Stations: 2
Thousandsticks Volunteer Fire & Rescue Department
Matched Stations: 0
Adrian Fire Department
Matched Stations: 6
Arab Fire Department
Matched Stations: 8
Corning Fire Department
Matched Stations: 14
Anaheim Fire & Rescue
Matched Stations: 24
Apple Valley Fire Protection District
Matched Stations: 16
Utah County Fire Department
Matched Stations: 4
Linton Township Volunteer Fire Protection Company  Inc.
Matched Stations: 4
Fountain Run Volunteer Fire Department
Matched Stations: 2
Jenkins Fire Department
Matched Stations: 2
Van

In [ ]:
queryUnmatchedStations = """
select a.name as firestation_name, ST_ASTEXT(ST_TRANSFORM(d.geom,900913)) as firestation_location
from firestation_usgsstructuredata a
inner join firestation_firestation b
  on a.id=b.usgsstructuredata_ptr_id
join firecares_core_address d 
  on b.station_address_id=d.id
where b.department_id is null and d.geom is not null
"""
#Query fireDepartments within 50 miles of station
queryNearbyDepartments = """
select c.name as department_name, c.id as dep_id
from firestation_firedepartment c 
inner join firecares_core_address e 
  on c.headquarters_address_id=e.id
where e.geom is not null and
ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_GEOMFROMTEXT(%s,900913)) * 0.000621371 <= 50 and
levenshtein(c.name,%s) <= 50; 
"""

queryCorellated = """
select a.name as firestation_name,
(
    select results.name
    from
    (
    select c.name, 
    ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_TRANSFORM(d.geom,900913)) * 0.000621371 as dis_miles,
    levenshtein(c.name,a.name) as dis_name
    from firestation_firedepartment c 
    inner join firecares_core_address e 
      on c.headquarters_address_id=e.id
    where e.geom is not null and
    ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_TRANSFORM(d.geom,900913)) * 0.000621371 <= 50 and
    levenshtein(c.name,a.name) <= 50
    order by dis_miles asc, dis_name asc
    ) as results 
    limit 1
) as department_name
from firestation_usgsstructuredata a
inner join firestation_firestation b
  on a.id=b.usgsstructuredata_ptr_id
join firecares_core_address d 
  on b.station_address_id=d.id
where b.department_id is null and d.geom is not null
"""
cursor.execute(queryCorellated)
fireStations = dictfetchall(cursor)
df = DataFrame(fireStations)
for fireStation in fireStations
    print "Fire Station Name:",fireStation['firestation_name']
    print "Department Name:",fireStation['department_name']

# distribution of levenshtein distances
#pl.xlabel("Levenshtein Distance")
#pl.ylabel("Count of records")
#totalMatchedDepartments = 0;
#for fireStation in fireStations:
#    params = [fireStation['firestation_location'],fireStation['firestation_name']]
#    reults = cursor.execute(queryNearbyDepartments,params)
#    nearbyDepartments = dictfetchall(cursor)
#    print "Matched Departments:", cursor.rowcount
#    totalMatchedDepartments += cursor.rowcount

#print "Total Matched Departments:", totalMatchedDepartments